In [18]:
import pandas as pd
import numpy as np
import geopandas as gpd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
from sklearn.inspection import PartialDependenceDisplay
from tqdm import tqdm  # 用于显示进度
# from sklearn.inspection import partial_dependence
from datetime import datetime
import os
import glob

best parameters by scenario

In [19]:
# Get all files under the directory
files = glob.glob('output/grid_search/*')
rows = []
for file in files:
    df_tmp = pd.read_csv(file)
    # Get the first row's first 4 columns
    first_row = df_tmp.iloc[0, 1:4].tolist()
    # Add file name and the 4 columns
    rows.append([os.path.basename(file)] + first_row)
# Create dataframe
result_df = pd.DataFrame(rows, columns=['file_name', 'tree_complexity', 'learning_rate', 'subsample'])
# Split 'file_name' into four columns based on '_'
def split_file_name(file_name):
    name = file_name.replace('.csv', '')
    parts = name.split('_')
    if 'Current' in parts[1]:
        bee = parts[0]
        scenario=parts[1]
        period = None
    else:
        bee = parts[0]
        scenario = parts[1]
        period = parts[2]
    return pd.Series([scenario, period, bee])
result_df[['scenario', 'period', 'bee']] = result_df['file_name'].apply(split_file_name)
result_df=result_df[['scenario', 'period','bee', 'tree_complexity', 'learning_rate', 'subsample']]

In [20]:
result_df

,scenario,period,bee,tree_complexity,learning_rate,subsample
0,Current,None,Apis cerana,3.0,0.001,0.70
1,ssp126,2041-2060,Apis cerana,3.0,0.001,0.70
2,ssp126,2061-2080,Apis cerana,3.0,0.001,0.70
3,ssp126,2081-2100,Apis cerana,3.0,0.001,0.70
4,ssp245,2041-2060,Apis cerana,3.0,0.001,0.75
5,ssp245,2061-2080,Apis cerana,3.0,0.001,0.80
6,ssp245,2081-2100,Apis cerana,3.0,0.001,0.70
7,ssp370,2041-2060,Apis cerana,3.0,0.001,0.70
8,ssp370,2061-2080,Apis cerana,3.0,0.001,0.75
9,ssp370,2081-2100,Apis cerana,3.0,0.001,0.75


predict probability

In [ ]:
def predict(given_scenario, given_year, given_bee, given_treecomplexity, given_learningrate, given_subsample):
    def label_status(row):
        if row['gb'] in case_county_list:
            return 1
        elif row['gb'] in all_surveyed_county_list:
            return 0
        else:
            return -999
    
    #risk factors
    if given_scenario=='Current':
        data=pd.read_csv(f'output/bee/risk_factor/Current.csv')
    else:
        data=pd.read_csv(f'output/bee/risk_factor/{given_scenario}_{given_year}.csv')
    risk_factors=data.columns[1:]

    #bee data
    bee_data=pd.read_excel(r'../CLEAN/output/MERGED_CLEAN.xlsx')
    all_surveyed_county_list=np.unique(bee_data['gb'])
    case_county_df=bee_data[bee_data['host_species']==given_bee].reset_index(drop=True)
    case_county_list=np.unique(case_county_df['gb'])

    data[given_bee] = data.apply(label_status, axis=1)
    survey_data=data[data[given_bee]!=-999].reset_index(drop=True)

    #add weight
    if given_scenario=='Current':
        weight=pd.read_csv(f'output/bee/weight/Current.csv')
    else:
        weight=pd.read_csv(f'output/bee/weight/{given_scenario}_{given_year}.csv')
    weight=weight[['gb','rescaled']]

    survey_data=survey_data.merge(weight, left_on=['gb'], right_on=['gb'], how='left')
    
    #step 1, identify key risk factors
    rel_con=pd.DataFrame({'Key': risk_factors})   
    for i in range(10):
            train, test = train_test_split(survey_data, test_size=0.25,  random_state=i)
            while test[test[given_bee]==1].shape[0]<1 or train[train[given_bee]==1].shape[0]<1:
                train, test = train_test_split(survey_data, test_size=0.25)
            train_x = train[risk_factors]
            train_y = train[given_bee]
            train_weight =  train['rescaled']
            model = xgb.XGBClassifier(objective='binary:logistic', learning_rate=given_learningrate, max_depth=given_treecomplexity, subsample=given_subsample, n_estimators=3000,  random_state=i) # colsample_bytree=1
            model.fit(train_x, train_y, sample_weight=train_weight)
            importances = model.get_booster().get_score(importance_type='weight')
            importances_df=pd.DataFrame(list(importances.items()), columns=['Key', 'Value'])
            importances_df=importances_df.rename(columns={'Value': 'Value_%d'%i})
            rel_con=rel_con.merge(importances_df, left_on='Key', right_on='Key', how='left')
    rel_con['sum'] = rel_con.iloc[:, 1:].sum(axis=1)
    rel_con['rc']=rel_con['sum']/rel_con['sum'].sum()
    key_risk_df=rel_con[rel_con['rc']>=0.05].reset_index(drop=True)
    key_risk_factors=key_risk_df['Key'].values
    
    #step 2, predict use key risk factors
    rel_con=pd.DataFrame({'Key': key_risk_factors})
    preds=[]
    auc_value=[]
    tpr_value=[]
    tnr_value=[]
    high_risk=[]
    for i in range(100):
        train, test = train_test_split(survey_data, test_size=0.25,  random_state=i)
        while test[test[given_bee]==1].shape[0]<1 or train[train[given_bee]==1].shape[0]<1:
            train, test = train_test_split(survey_data, test_size=0.25)

        train_x = train[key_risk_factors]
        train_y = train[given_bee]
        train_weight =  train['rescaled']
        model = xgb.XGBClassifier(objective='binary:logistic', learning_rate=given_learningrate, max_depth=given_treecomplexity, subsample=given_subsample, n_estimators=3000,  random_state=i) # colsample_bytree=1
        model.fit(train_x, train_y, sample_weight=train_weight)
        importances = model.get_booster().get_score(importance_type='weight')
        importances_df=pd.DataFrame(list(importances.items()), columns=['Key', 'Value'])
        importances_df=importances_df.rename(columns={'Value': 'Value_%d'%i})
        rel_con=rel_con.merge(importances_df, left_on='Key', right_on='Key', how='left')
        
        test_x = test[key_risk_factors]
        test_y = test[given_bee]
        pred_test_y=model.predict_proba(test_x)[:,1]
        fpr, tpr, thresholds = roc_curve(test_y, pred_test_y)
        roc_auc = auc(fpr, tpr)
        auc_value.append(roc_auc)
        optimal_idx = np.argmax(tpr - fpr)  # 找到最大约登指数对应的索引
        optimal_threshold = thresholds[optimal_idx]  # 最佳阈值
        optimal_fpr = fpr[optimal_idx]  # 最佳阈值对应的FPR
        optimal_tpr = tpr[optimal_idx]  # 最佳阈值对应的TPR
        tpr_value.append(optimal_tpr)
        tnr_value.append(1-optimal_fpr)       

        all_x=data[key_risk_factors]
        pred_all_y=model.predict_proba(all_x)[:,1]
        pred_df=pd.DataFrame({'gb':data['gb'], 'round': [i]*data.shape[0], 'status':data[given_bee],'pred':pred_all_y,'sensitivity':[optimal_tpr]*data.shape[0],'specificity': [(1-optimal_fpr)]*data.shape[0]})
        preds.append(pred_df)

        high_risk_data=pred_df[pred_df['pred']>=0.4].reset_index(drop=True)
        high_risk_data['threshold']=[optimal_threshold]*high_risk_data.shape[0]
        high_risk.append(high_risk_data)

    #mean auc of testing set
    auc_mean=np.mean(auc_value)
    auc_ubd=np.percentile(auc_value, 97.5)
    auc_lbd=np.percentile(auc_value, 2.5)
    auc_df=pd.DataFrame({'index': ['mean', 'lbd', 'ubd'], 'value': [auc_mean, auc_lbd, auc_ubd]})
    # Ensure output directory exists
    output_folder = 'output/bee/auc'
    os.makedirs(output_folder, exist_ok=True)
    if given_scenario=='Current':
        auc_df.to_csv(f'{output_folder}/Current_{given_bee}.csv', index=False)
    else:
        auc_df.to_csv(f'{output_folder}/{given_scenario}_{given_year}_{given_bee}.csv', index=False)
        
    tpr_mean=np.mean(tpr_value)
    tpr_ubd=np.percentile(tpr_value, 97.5)
    tpr_lbd=np.percentile(tpr_value, 2.5)
    tpr_df=pd.DataFrame({'index': ['mean', 'lbd', 'ubd'], 'value': [tpr_mean, tpr_lbd, tpr_ubd]})
    output_folder = 'output/bee/tpr'
    os.makedirs(output_folder, exist_ok=True)
    if given_scenario=='Current':
        tpr_df.to_csv(f'{output_folder}/Current_{given_bee}.csv', index=False)
    else:
        tpr_df.to_csv(f'{output_folder}/{given_scenario}_{given_year}_{given_bee}.csv', index=False)

    tnr_mean=np.mean(tnr_value)
    tnr_ubd=np.percentile(tnr_value, 97.5)
    tnr_lbd=np.percentile(tnr_value, 2.5)
    tnr_df=pd.DataFrame({'index': ['mean', 'lbd', 'ubd'], 'value': [tnr_mean, tnr_lbd, tnr_ubd]})
    output_folder = 'output/bee/tnr'
    os.makedirs(output_folder, exist_ok=True)
    if given_scenario=='Current':
        tnr_df.to_csv(f'{output_folder}/Current_{given_bee}.csv', index=False)
    else:
        tnr_df.to_csv(f'{output_folder}/{given_scenario}_{given_year}_{given_bee}.csv', index=False)


    #relative contribution
    rel_con['row_sum']=rel_con.iloc[:, 1:].sum(axis=1)
    rel_con['std_sum']=rel_con['row_sum']/rel_con['row_sum'].sum()
    output_folder = 'output/bee/rc'
    os.makedirs(output_folder, exist_ok=True)
    if given_scenario=='Current':
        rel_con.to_csv(f'{output_folder}/Current_{given_bee}.csv', index=False)
    else:
        rel_con.to_csv(f'{output_folder}/{given_scenario}_{given_year}_{given_bee}.csv', index=False)

    #high_risk_counties
    high_risk_df=pd.concat(high_risk, ignore_index=True)
    output_folder = 'output/bee/hrc'
    os.makedirs(output_folder, exist_ok=True)
    if given_scenario=='Current':
        high_risk_df.to_csv(f'{output_folder}/Current_{given_bee}.csv', index=False)
    else:
        high_risk_df.to_csv(f'{output_folder}/{given_scenario}_{given_year}_{given_bee}.csv', index=False)

    #calcualte mean, std and 95% CI
    pred_df=pd.concat(preds, ignore_index=True)
    grouped = pred_df.groupby(['gb', 'status'])['pred'].mean()
    grouped = grouped.reset_index()
    output_folder = 'output/bee/prob'
    os.makedirs(output_folder, exist_ok=True)
    if given_scenario=='Current':
        grouped.to_csv(f'{output_folder}/Current_{given_bee}.csv', index=False)
    else:
        grouped.to_csv(f'{output_folder}/{given_scenario}_{given_year}_{given_bee}.csv', index=False)

loop all scenarios

In [ ]:
for idx, row in result_df.iterrows():
    scenario = row['scenario']
    period = row['period']
    bee = row['bee']
    tree_complexity = int(row['tree_complexity'])
    learning_rate = float(row['learning_rate'])
    subsample = float(row['subsample'])
    # 这里可以根据需要对每一组参数进行操作
    print(scenario, period, bee, tree_complexity, learning_rate, subsample)
    predict(given_scenario=scenario, given_year=period, given_bee=bee, given_treecomplexity=tree_complexity, given_learningrate=learning_rate, given_subsample=subsample)

combine AUC

In [ ]:
df=pd.DataFrame({'index': ['mean', 'lbd', 'ubd']})
for idx, row in result_df.iterrows():
    scenario = row['scenario']
    period = row['period']
    bee = row['bee']
    if scenario=='Current':
        tmp=pd.read_csv(f'output/bee/auc/Current_{bee}.csv')
        df[f'{scenario}_{bee}']=tmp['value']
    else:
        tmp=pd.read_csv(f'output/bee/auc/{scenario}_{period}_{bee}.csv')
        df[f'{scenario}_{period}_{bee}']=tmp['value']
dfn=df.T
output_folder = 'output/bee/auc'
os.makedirs(output_folder, exist_ok=True)
dfn.to_csv(f'{output_folder}/Combine.csv')    

combine True positive rate

In [ ]:
df=pd.DataFrame({'index': ['mean', 'lbd', 'ubd']})
for idx, row in result_df.iterrows():
    scenario = row['scenario']
    period = row['period']
    bee = row['bee']
    if scenario=='Current':
        tmp=pd.read_csv(f'output/bee/tpr/Current_{bee}.csv')
        df[f'{scenario}_{bee}']=tmp['value']
    else:
        tmp=pd.read_csv(f'output/bee/tpr/{scenario}_{period}_{bee}.csv')
        df[f'{scenario}_{period}_{bee}']=tmp['value']
dfn=df.T
output_folder = 'output/bee/tpr'
os.makedirs(output_folder, exist_ok=True)
dfn.to_csv(f'{output_folder}/Combine.csv')

combine true negative rate

In [ ]:
df=pd.DataFrame({'index': ['mean', 'lbd', 'ubd']})
for idx, row in result_df.iterrows():
    scenario = row['scenario']
    period = row['period']
    bee = row['bee']
    if scenario=='Current':
        tmp=pd.read_csv(f'output/bee/tnr/Current_{bee}.csv')
        df[f'{scenario}_{bee}']=tmp['value']
    else:
        tmp=pd.read_csv(f'output/bee/tnr/{scenario}_{period}_{bee}.csv')
        df[f'{scenario}_{period}_{bee}']=tmp['value']
dfn=df.T
output_folder = 'output/bee/tnr'
os.makedirs(output_folder, exist_ok=True)
dfn.to_csv(f'{output_folder}/Combine.csv')

combine relative contribution

In [ ]:
df=pd.DataFrame({'factor': ['Bio1','Bio2','Bio3','Bio4','Bio5','Bio6','Bio7','Bio8','Bio9','Bio10','Bio11','Bio12','Bio13','Bio14','Bio15','Bio16','Bio17','Bio18','Bio19', 'average_elevation', 
                            'proportion_10', 'proportion_20','proportion_30', 'proportion_40', 'proportion_50', 'proportion_60',
                            'proportion_70', 'proportion_80','proportion_90', 'proportion_95', 'proportion_100']})
for idx, row in result_df.iterrows():
    scenario = row['scenario']
    period = row['period']
    bee = row['bee']
    if scenario=='Current':
        tmp=pd.read_csv(f'output/bee/rc/Current_{bee}.csv')
        tmp=tmp[['Key', 'std_sum']]
        tmp=tmp.rename(columns={'Key': 'factor', 'std_sum': f'{scenario}_{bee}'})
    else:
        tmp=pd.read_csv(f'output/bee/rc/{scenario}_{period}_{bee}.csv')
        tmp=tmp[['Key', 'std_sum']]
        tmp=tmp.rename(columns={'Key': 'factor', 'std_sum': f'{scenario}_{period}_{bee}'})
    df=df.merge(tmp, left_on='factor', right_on='factor', how='left')
output_folder = 'output/bee/rc'
os.makedirs(output_folder, exist_ok=True)
df.to_csv(f'{output_folder}/Combine.csv')

combine high risk counties

In [21]:
obv_number=[]
obv_area=[]
pred_number=[]
pred_area=[]
col_bee=[]

for idx, row in result_df.iterrows():
    scenario = row['scenario']
    period = row['period']
    bee = row['bee']
    if scenario=='Current':
        pred=pd.read_csv(f'output/prob/Current_{bee}.csv')
        pred=pred[pred['pred']>=0.4].reset_index(drop=True)
    else:
        pred=pd.read_csv(f'output/prob/{scenario}_{period}_{bee}.csv')
        pred=pred[pred['pred']>=0.4].reset_index(drop=True)
    high_risk_county=np.unique(pred['gb'])
    
    bee_data=pd.read_excel(r'../../CLEAN/output/MERGED_CLEAN.xlsx')
    all_surveyed_county_list=np.unique(bee_data['gb'])
    case_county_df=bee_data[bee_data['host_species']==bee].reset_index(drop=True)
    case_county_list=np.unique(case_county_df['gb'])
    
    county_information=pd.read_excel(r'input/China_county_level_population_area.xlsx')

    number=0
    area=0
    for j in range(len(high_risk_county)):
        gb=high_risk_county[j]
        tmp=county_information[county_information['gb']==gb].reset_index(drop=True)
        if tmp.shape[0]!=1:
            print('pred', tmp)
        else:
            number+=1
            area+=(tmp['area(km2)'][0])
    pred_number.append(number)
    pred_area.append(area)

    number=0
    area=0
    for j in range(len(case_county_list)):
        gb=case_county_list[j]
        tmp=county_information[county_information['gb']==gb].reset_index(drop=True)
        if tmp.shape[0]!=1:
            print('pred', tmp)
        else:
            number+=1
            area+=(tmp['area(km2)'][0])
    obv_number.append(number)
    obv_area.append(area) 
    
    if scenario=='Current':
        col_bee.append('Current_'+bee)
    else:
        col_bee.append(f'{scenario}_{period}_{bee}')

col_1=[]
col_2=[]
for i in range(len(col_bee)):
    obv1=obv_number[i]
    pred1=pred_number[i]
    ratio1=100*(pred1-obv1)/obv1
    # print(col_bee[i], obv1, pred1, ratio1)
    col_1.append("%d/%d (%.1f)"%(pred1, obv1, ratio1))

    obv2=obv_area[i]/1e6
    pred2=pred_area[i]/1e6
    ratio2=100*(pred2-obv2)/obv2
    col_2.append("%.2f/%.2f (%.1f)"%(pred2, obv2, ratio2))

dfn=pd.DataFrame({'bee': col_bee, 'county': col_1, 'area': col_2})
output_folder = 'output/bee/pc'
os.makedirs(output_folder, exist_ok=True)
dfn.to_csv(f'{output_folder}/Combine.csv', index=False)         